In [16]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [19]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///aid_data.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

aid = Base.classes.aid_data

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [ ]:
year = 2016

In [23]:
sel = [
        aid.country_name,
        aid.latitude,
        aid.longitude,
        aid.transaction_type_name,
        aid.constant_amount,
        aid.fiscal_year,        
    ]

results = db.session.query(*sel).filter(aid.fiscal_year == year).group_by(aid.country_name, aid.fiscal_year, aid.transaction_type_name).all()

# Create a dictionary entry for each row of metadata information
aid_dict = {}
for result in results:
    aid_dict["country"] = result[0]
    aid_dict["latitude"] = result[1]
    aid_dict["longitude"] = result[2]
    aid_dict["transaction type"] = result[3]
    aid_dict["amount"] = result[4]
    aid_dict["year"] = result[5]
print(aid_dict)

{'country': 'Zimbabwe', 'latitude': -19.015438, 'longitude': 29.154857, 'transaction type': 'Obligations', 'amount': -119269, 'year': 2016}


In [22]:
stmt = db.session.query(aid).statement
df = pd.read_sql_query(stmt, db.session.bind)
df.head()

,index,country_code,country_name,latitude,longitude,region_name,income_group_name,income_group_acronym,implementing_agency_acronym,implementing_agency_name,...,activity_project_number,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,current_amount,constant_amount,USG_sector_name,submission_id
0,0,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2011,9941000000,10731991839,Stabilization Operations and Security Sector R...,28
1,1,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2012,9243000000,9799467226,Stabilization Operations and Security Sector R...,28
2,2,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,3,Disbursements,2011,7840175215,8464007285,Stabilization Operations and Security Sector R...,28
3,3,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,3,Disbursements,2013,7764310985,8095048196,Stabilization Operations and Security Sector R...,28
4,4,AFG,Afghanistan,33.93911,67.709953,South and Central Asia,Low Income Country,LIC,DOD,Department of Defense,...,None,None,None,2,Obligations,2013,6928000000,7223112779,Stabilization Operations and Security Sector R...,28


In [35]:
# Filter the data based on the year
year = 2016

aid_data = df.loc[(df["fiscal_year"] == year), ["country_name", "latitude", "longitude", "transaction_type_name", "constant_amount", "fiscal_year"]]
# Format the data to send as json
aid_data.head()



,country_name,latitude,longitude,transaction_type_name,constant_amount,fiscal_year
22,Afghanistan,33.939110,67.709953,Obligations,3754000000,2016
33,Israel,31.046051,34.851612,Obligations,3100000000,2016
34,Israel,31.046051,34.851612,Disbursements,3100000000,2016
47,Iraq,33.223191,43.679291,Disbursements,2850000000,2016
48,Iraq,33.223191,43.679291,Obligations,2850000000,2016


In [25]:
aid_by_country = aid_data.groupby(["country_name", "latitude", "longitude", "fiscal_year", "transaction_type_name"])["constant_amount"].sum()
aid_by_country_df = pd.DataFrame(aid_by_country)
aid_by_country_df.head()

constant_amount
country_name latitude  longitude fiscal_year transaction_type_name                 
Afghanistan  33.939110 67.709953 2016        Disbursements               4223684237
                                             Obligations                 5060084350
Albania      41.153332 20.168331 2016        Disbursements                 28703594
                                             Obligations                   27459364
Algeria      28.033886 1.659626  2016        Disbursements                  8341056

In [28]:
map_data = aid_by_country_df.to_dict('records')
map_data

[{'constant_amount': 4223684237},
 {'constant_amount': 5060084350},
 {'constant_amount': 28703594},
 {'constant_amount': 27459364},
 {'constant_amount': 8341056},
 {'constant_amount': 17641406},
 {'constant_amount': 69153127},
 {'constant_amount': 67149547},
 {'constant_amount': 635781},
 {'constant_amount': 635781},
 {'constant_amount': 6148126},
 {'constant_amount': 2439204},
 {'constant_amount': 46024007},
 {'constant_amount': 22205304},
 {'constant_amount': 2538},
 {'constant_amount': 2538},
 {'constant_amount': 71269},
 {'constant_amount': 71269},
 {'constant_amount': 24501071},
 {'constant_amount': 15232389},
 {'constant_amount': 2694794},
 {'constant_amount': 3313446},
 {'constant_amount': 7149660},
 {'constant_amount': 6573352},
 {'constant_amount': 266099370},
 {'constant_amount': 263396622},
 {'constant_amount': 4244751},
 {'constant_amount': 5442370},
 {'constant_amount': 10646578},
 {'constant_amount': 9226608},
 {'constant_amount': 7073320},
 {'constant_amount': 8613838},
